# **Grid World Problem**
---



> a classical problem in reinforcement learning where an agent must navigate a grid to reach a goal state while avoiding obstacles. The objective is to find the optimal policy that maximizes the cumulative reward. Now I will implement this problem with the 5 different algorithms shown below.


1. Value Iteration


A dynamic programming algorithm used to compute the optimal value function by iteratively improving the value estimates.

Let's use a small 3x4 grid as an example where:

- 'S' denotes the start state.
- 'G' denotes the goal state.
- 'O' denotes obstacles.
- ' ' denotes empty cells.

In [13]:
import numpy as np
import random

def value_iteration(grid, rewards, gamma=0.9, theta=1e-6):
    n, m = grid.shape
    V = np.zeros((n, m))
    while True:
        delta = 0
        for i in range(n):
            for j in range(m):
                if grid[i, j] in ['G', 'O']:
                    continue
                v = V[i, j]
                V[i, j] = max([
                    rewards[i, j] + gamma * V[min(n-1, i+1), j],  # Down
                    rewards[i, j] + gamma * V[max(0, i-1), j],    # Up
                    rewards[i, j] + gamma * V[i, min(m-1, j+1)],  # Right
                    rewards[i, j] + gamma * V[i, max(0, j-1)]     # Left
                ])
                delta = max(delta, abs(v - V[i, j]))
        if delta < theta:
            break
    return V

def main():
    grid = np.array([['S', ' ', ' ', 'G'],
                     [' ', 'O', ' ', ' '],
                     [' ', ' ', ' ', ' ']])
    rewards = np.array([[-1, -1, -1, 10],
                        [-1, -10, -1, -1],
                        [-1, -1, -1, -1]])
    V = value_iteration(grid, rewards)
    print("Optimal Value Function:")
    print(V)

if __name__ == "__main__":
    main()


Optimal Value Function:
[[-1.9 -1.  -1.   0. ]
 [-1.   0.  -1.  -1. ]
 [-1.9 -1.  -1.9 -1.9]]


2. Policy Iteration

Another dynamic programming algorithm that alternates between policy evaluation and policy improvement to find the optimal policy.


In [12]:
def policy_evaluation(policy, grid, rewards, gamma=0.9, theta=1e-6):
    n, m = grid.shape
    V = np.zeros((n, m))
    while True:
        delta = 0
        for i in range(n):
            for j in range(m):
                if grid[i, j] in ['G', 'O']:
                    continue
                v = V[i, j]
                action = policy[i, j]
                if action == 0:  # Down
                    V[i, j] = rewards[i, j] + gamma * V[min(n-1, i+1), j]
                elif action == 1:  # Up
                    V[i, j] = rewards[i, j] + gamma * V[max(0, i-1), j]
                elif action == 2:  # Right
                    V[i, j] = rewards[i, j] + gamma * V[i, min(m-1, j+1)]
                elif action == 3:  # Left
                    V[i, j] = rewards[i, j] + gamma * V[i, max(0, j-1)]
                delta = max(delta, abs(v - V[i, j]))
        if delta < theta:
            break
    return V

def policy_iteration(grid, rewards, gamma=0.9):
    n, m = grid.shape
    policy = np.random.choice(4, size=(n, m))
    while True:
        V = policy_evaluation(policy, grid, rewards)
        policy_stable = True
        for i in range(n):
            for j in range(m):
                if grid[i, j] in ['G', 'O']:
                    continue
                old_action = policy[i, j]
                policy[i, j] = np.argmax([
                    rewards[i, j] + gamma * V[min(n-1, i+1), j],  # Down
                    rewards[i, j] + gamma * V[max(0, i-1), j],    # Up
                    rewards[i, j] + gamma * V[i, min(m-1, j+1)],  # Right
                    rewards[i, j] + gamma * V[i, max(0, j-1)]     # Left
                ])
                if old_action != policy[i, j]:
                    policy_stable = False
        if policy_stable:
            break
    return policy, V

grid = np.array([['S', ' ', ' ', 'G'],
                  [' ', 'O', ' ', ' '],
                  [' ', ' ', ' ', ' ']])
rewards = np.array([[-1, -1, -1, 10],
                    [-1, -10, -1, -1],
                    [-1, -1, -1, -1]])
policy, V = policy_iteration(grid, rewards)
print("Optimal Policy:")
print(policy)
print("Optimal Value Function:")
print(V)

Optimal Policy:
[[0 0 2 1]
 [2 3 3 1]
 [1 1 1 1]]
Optimal Value Function:
[[-1.9 -1.  -1.   0. ]
 [-1.   0.  -1.  -1. ]
 [-1.9 -1.  -1.9 -1.9]]


3. Q-Learning


A model-free RL algorithm that learns the action-value function.

In [11]:
def q_learning(grid, rewards, alpha=0.1, gamma=0.9, epsilon=0.1, episodes=1000):
    n, m = grid.shape
    Q = np.zeros((n, m, 4))

    for _ in range(episodes):
        i, j = 0, 0  # Start state
        while grid[i, j] != 'G':
            if random.uniform(0, 1) < epsilon:
                action_idx = np.random.choice(4)
            else:
                action_idx = np.argmax(Q[i, j])
            action = action_list[action_idx]

            next_i, next_j = next_state((i, j), action)
            reward = rewards[next_i, next_j]
            Q[i, j, action_idx] = Q[i, j, action_idx] + alpha * (reward + gamma * np.max(Q[next_i, next_j]) - Q[i, j, action_idx])

            i, j = next_i, next_j

    return Q


# Define the environment
grid = np.array([['S', ' ', ' ', 'G'],
                 [' ', 'O', ' ', ' '],
                 [' ', ' ', ' ', ' ']])

# Define rewards
rewards = np.array([[-1, -1, -1, 10],
                    [-1, -10, -1, -1],
                    [-1, -1, -1, -1]])

# Define actions
actions = {'up': 0, 'down': 1, 'left': 2, 'right': 3}
action_list = ['up', 'down', 'left', 'right']

# Define state transitions
def next_state(state, action):
    i, j = state
    if action == 'up':
        return max(0, i-1), j
    elif action == 'down':
        return min(grid.shape[0]-1, i+1), j
    elif action == 'left':
        return i, max(0, j-1)
    elif action == 'right':
        return i, min(grid.shape[1]-1, j+1)

Q = q_learning(grid, rewards)
print("Q-Table:")
print(Q)



Q-Table:
[[[ 4.4706951   1.98361605  4.2500508   6.2       ]
  [ 5.71981895 -6.20951597  4.04018704  8.        ]
  [ 7.77337752  5.11080461  5.48634996 10.        ]
  [ 0.          0.          0.          0.        ]]

 [[ 4.1440519  -0.5318209  -0.58519851 -2.99761474]
  [-0.19       -0.3138499  -0.02599529  5.20180598]
  [ 7.94393946 -0.12032709 -1.64846397  0.1997    ]
  [ 3.439       0.          0.          0.        ]]

 [[-0.41935677 -0.3940399  -0.3940399  -0.54327349]
  [-1.         -0.3940399  -0.43038784 -0.38858453]
  [ 1.38470587 -0.199      -0.109      -0.1       ]
  [-0.1         0.          0.          0.        ]]]


4. Epsilon-Greedy Policy

A policy that balances exploration and exploitation by choosing a random action with probability epsilon and the best-known action with probability 1-epsilon.

In [10]:
def epsilon_greedy(Q, state, epsilon):
    if random.uniform(0, 1) < epsilon:
        return np.random.choice(4)
    else:
        return np.argmax(Q[state])

grid = np.array([['S', ' ', ' ', 'G'],
                  [' ', 'O', ' ', ' '],
                  [' ', ' ', ' ', ' ']])
rewards = np.array([[-1, -1, -1, 10],
                    [-1, -10, -1, -1],
                    [-1, -1, -1, -1]])
epsilon = 0.1
Q = q_learning(grid, rewards)  # Use Q-learning to get Q-values
state = (0, 0)
action = epsilon_greedy(Q, state, epsilon)
print("Chosen Action:", action)


Chosen Action: 3


5. UCB Algorithm

An algorithm that balances exploration and exploitation using an upper confidence bound.

In [9]:
def q_learning_ucb(grid, rewards, alpha=0.1, gamma=0.9, episodes=1000):
    n, m = grid.shape
    Q = np.zeros((n, m, 4))
    N = np.zeros((n, m, 4))

    for t in range(1, episodes+1):
        i, j = 0, 0  # Start state
        while grid[i, j] != 'G':
            action_idx = ucb(Q, N, (i, j), t)
            N[i, j, action_idx] += 1
            action = action_list[action_idx]

            next_i, next_j = next_state((i, j), action)
            reward = rewards[next_i, next_j]
            Q[i, j, action_idx] = Q[i, j, action_idx] + alpha * (reward + gamma * np.max(Q[next_i, next_j]) - Q[i, j, action_idx])

            i, j = next_i, next_j

    return Q

# Define the environment
grid = np.array([['S', ' ', ' ', 'G'],
                 [' ', 'O', ' ', ' '],
                 [' ', ' ', ' ', ' ']])

# Define rewards
rewards = np.array([[-1, -1, -1, 10],
                    [-1, -10, -1, -1],
                    [-1, -1, -1, -1]])

# Define actions
actions = {'up': 0, 'down': 1, 'left': 2, 'right': 3}
action_list = ['up', 'down', 'left', 'right']

# Define state transitions
def next_state(state, action):
    i, j = state
    if action == 'up':
        return max(0, i-1), j
    elif action == 'down':
        return min(grid.shape[0]-1, i+1), j
    elif action == 'left':
        return i, max(0, j-1)
    elif action == 'right':
        return i, min(grid.shape[1]-1, j+1)

# UCB implementation
def ucb(Q, N, state, t):
    ucb_values = Q[state] + np.sqrt(2 * np.log(t + 1) / (N[state] + 1e-5))
    return np.argmax(ucb_values)



Q = q_learning_ucb(grid, rewards)
print("Q-Table:")
print(Q)

Q-Table:
[[[-0.76616999 -0.76565225 -0.76616999  6.2       ]
  [-0.29539    -1.909      -0.3277009   8.        ]
  [-0.1        -0.1        -0.109      10.        ]
  [ 0.          0.          0.          0.        ]]

 [[-0.62963786 -0.5396941  -0.58519851 -1.9       ]
  [-0.20629    -0.19981    -0.109      -0.19171   ]
  [ 0.178559   -0.1        -1.009      -0.019     ]
  [ 1.9        -0.1        -0.1009     -0.01      ]]

 [[-0.4107114  -0.3940399  -0.3940399  -0.46468   ]
  [-1.909      -0.29701    -0.3204109  -0.28      ]
  [-0.19       -0.199      -0.109      -0.19      ]
  [-0.1        -0.1        -0.109      -0.1       ]]]


# **Single-State Multi-Armed Bandit Problem**

---



> a reinforcement learning problem where an agent chooses from
𝐾
K actions (or "arms"), each providing a stochastic reward, to maximize its cumulative reward over time. The challenge is to balance exploring new actions and exploiting known high-reward actions to develop a strategy that maximizes total reward.



1. Value Iteration

Value Iteration is typically not used for multi-armed bandit problems as it is more suited for Markov Decision Processes (MDPs). It Computes the optimal value function by iteratively updating value estimates based on the expected rewards from each arm.

In [14]:
import numpy as np

def value_iteration_bandit(rewards, gamma=0.9, theta=1e-6):
    K = len(rewards)
    V = np.zeros(K)
    while True:
        delta = 0
        for k in range(K):
            v = V[k]
            V[k] = rewards[k] + gamma * np.max(V)
            delta = max(delta, abs(v - V[k]))
        if delta < theta:
            break
    return V

rewards = np.random.normal(0, 1, 10)  # Assume 10 arms with normal rewards
V = value_iteration_bandit(rewards)
print("Optimal Value Function:")
print(V)


main()


Optimal Value Function:
[12.79918108 13.30276888 14.62488109 14.57650006 13.22041244 13.63134782
 14.58398371 14.73100793 15.63180703 14.13623874]
Optimal Value Function:
[[-1.9 -1.  -1.   0. ]
 [-1.   0.  -1.  -1. ]
 [-1.9 -1.  -1.9 -1.9]]


2. Policy Iteration

Alternates between policy evaluation and policy improvement to find the optimal policy for selecting arms.

In [15]:
import numpy as np

def policy_evaluation_bandit(policy, rewards, gamma=0.9, theta=1e-6):
    K = len(rewards)
    V = np.zeros(K)
    while True:
        delta = 0
        for k in range(K):
            v = V[k]
            V[k] = rewards[k] + gamma * V[policy[k]]
            delta = max(delta, abs(v - V[k]))
        if delta < theta:
            break
    return V

def policy_iteration_bandit(rewards, gamma=0.9):
    K = len(rewards)
    policy = np.random.choice(K, K)
    while True:
        V = policy_evaluation_bandit(policy, rewards, gamma)
        policy_stable = True
        for k in range(K):
            old_action = policy[k]
            policy[k] = np.argmax(rewards + gamma * V)
            if old_action != policy[k]:
                policy_stable = False
        if policy_stable:
            break
    return policy, V


rewards = np.random.normal(0, 1, 10)  # Assume 10 arms with normal rewards
policy, V = policy_iteration_bandit(rewards)
print("Optimal Policy:")
print(policy)
print("Optimal Value Function:")
print(V)



Optimal Policy:
[3 3 3 3 3 3 3 3 3 3]
Optimal Value Function:
[14.29401254 12.80423241 14.01454389 15.34704073 14.01365561 15.11124093
 14.32990496 14.75615847 13.80987414 15.24004483]


3. Q-Learning

Learns the action-value function by updating Q-values based on the rewards received from selecting each arm.

In [16]:
def q_learning_bandit(K, rewards, alpha=0.1, gamma=0.9, epsilon=0.1, episodes=1000):
    Q = np.zeros(K)
    for _ in range(episodes):
        if random.uniform(0, 1) < epsilon:
            action = np.random.choice(K)
        else:
            action = np.argmax(Q)
        reward = rewards[action]
        Q[action] = Q[action] + alpha * (reward + gamma * np.max(Q) - Q[action])
    return Q

def main():
    rewards = np.random.normal(0, 1, 10)  # Assume 10 arms with normal rewards
    Q = q_learning_bandit(len(rewards), rewards)
    print("Q-Table:")
    print(Q)

if __name__ == "__main__":
    main()


Q-Table:
[ 6.71309595  3.34129343 11.65742196  6.06350381  4.98248368  6.78434214
  5.41057989  6.67700878  7.52827967  8.58454842]


4. Epsilon-Greedy Policy

Balances exploration and exploitation by selecting a random arm with probability epsilon and the best-known arm with probability 1-epsilon.

In [17]:
def epsilon_greedy_bandit(K, rewards, epsilon=0.1, episodes=1000):
    Q = np.zeros(K)
    N = np.zeros(K)
    for _ in range(episodes):
        if random.uniform(0, 1) < epsilon:
            action = np.random.choice(K)
        else:
            action = np.argmax(Q)
        reward = rewards[action]
        N[action] += 1
        Q[action] = Q[action] + (1 / N[action]) * (reward - Q[action])
    return Q

def main():
    rewards = np.random.normal(0, 1, 10)  # Assume 10 arms with normal rewards
    Q = epsilon_greedy_bandit(len(rewards), rewards)
    print("Q-Table:")
    print(Q)

if __name__ == "__main__":
    main()


Q-Table:
[-0.58892424 -1.44681539  1.01538722 -0.69413363 -1.41262326  0.66675352
  0.1621129   0.07888828  0.6576616   0.14150586]


5. UCB Algorithm

Balances exploration and exploitation using an upper confidence bound to select the arm with the highest potential reward.

In [18]:
def ucb_bandit(K, rewards, episodes=1000):
    Q = np.zeros(K)
    N = np.zeros(K)
    for t in range(1, episodes + 1):
        if t <= K:
            action = t - 1  # Initialize by playing each arm once
        else:
            ucb_values = Q + np.sqrt(2 * np.log(t) / (N + 1e-5))
            action = np.argmax(ucb_values)
        reward = rewards[action]
        N[action] += 1
        Q[action] = Q[action] + (1 / N[action]) * (reward - Q[action])
    return Q

def main():
    rewards = np.random.normal(0, 1, 10)  # Assume 10 arms with normal rewards
    Q = ucb_bandit(len(rewards), rewards)
    print("Q-Table:")
    print(Q)

if __name__ == "__main__":
    main()


Q-Table:
[-0.23692975  2.22726405  0.60906658  0.85371416  0.67962946  0.0858082
 -1.60707274 -0.01528898 -0.11568769  0.8332205 ]
